In [ ]:
# default_exp graph

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Graph module

> Provides main class and helpers classes and functions to handle pangenome graph and related data structures

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted 05_export.ipynb.
Converted deBruijnGraphProcessing.ipynb.
No export destination, ignored:
#exporti

import os
import glob
# import json
# import time
# import itertools
import warnings

# from collections.abc import Iterable

import pdb

# from copy import deepcopy

import numpy as np

# from sortedcontainers import SortedSet
# from multiprocessing import Process, Manager, cpu_count

# from redis import ResponseError

# from pangraph_constructor.utils import pathConvert,NpEncoder,adjustZoomLevels
# from pangraph_constructor.utils import iset_add,getDBID,resetDB
from pangraph_constructor.graph import GenomeGraph,initialPathAnalysis,calcNodeLengths,getPathNodeInversionRate
from pangraph_constructor.utils import pathFileToPathDict
Converted dev.ipynb.
Converted graphTesting.ipynb.
Converted index (2).ipynb.
Converted index.ipynb.


In [ ]:
#exporti

import pdb

import os
import glob
import json
import itertools
import warnings
import joblib

import networkx as nx

import pandas as pd
import numpy as np

from skbio.io import read as skbio_read
from skbio.metadata import IntervalMetadata
from skbio.sequence import DNA

from dnasim.IO import writeFASTA
from dnasim.simulation import inverseSequence

from pangraph_constructor.synteny import processAccession,generatePathsLinks
from pangraph_constructor.utils import bidict,NpEncoder,pathFileToPathDict
from pangraph_constructor.tree import TremauxTree

In [ ]:
#export
def calcNodeLengths(graph):
    print('Calculating nodes length...')
    
    numNodes = len(graph.nodes)
    numNodesDigits = np.int(np.ceil(np.log10(numNodes)))
    nodeLengths = [0]*numNodes
    
    for nodeIdx in range(numNodes):
        print(f'\nProcessing node {nodeIdx+1:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')
        if graph.nodesData[nodeIdx]=='':
            nodeLengths[nodeIdx] = len(graph.nodes[nodeIdx])
        else:
            nodeLengths[nodeIdx] = len(graph.nodesData[nodeIdx])
    
    print('\nFinished calculating nodes lengths')
    return nodeLengths

In [ ]:
#export
def initialPathAnalysis(graph,nodeLengths):
    '''
    
    Return
    ======
    `pathLengths`: TBD

    `pathNodeArray`: numpy.array[int]. An array of the size number_of_paths x max_length_of_path, 
                    containing sequence of nodes in the order as they are passed by the given 
                    path. If the length of the path is less than the number of columns of the
                    array, then the unused columns are padded with zeros (node numbers are 1-based).
    
    `pathNodeLengths`: TBD
    
    `pathDirArray`: numpy.array[bool]. An array of the size number_of_paths x max_length_of_path, 
                    containing direction (normal or inversed) in which the given path passes each node. 
                    `False` means normal direction, `True` means inverted. If the length of the path 
                    is less than the number of columns of the array, then the unused columns 
                    are padded with `False`.
    
    `pathNodeLengthsCum`: TBD
    
    
    Example
    =======
    
    The following paths are present in the graph (containing 5 nodes):
    seq1: 1+, 2+, 3+, 4+, 5+ 
    ... # seq2 - seq6 are the same as seq1 as inversion is relative to consensus
    seq7: 1+, 4-, 3-, 2-, 5+
    seq8: 1+, 4-, 3-, 2-, 4-, 3-, 2-, 5+
    seq9: 1+, 4-, 3+, 2-, 5+
    seq10: 1+, 4-, 3+, 2-, 4-, 3+, 2-, 5+
    
    The result is:
    `pathLengths`: TBD
    
    `pathNodeArray`:
    array([[1, 2, 3, 4, 5, 0, 0, 0],
           ...
           [1, 4, 3, 2, 5, 0, 0, 0],
           [1, 4, 3, 2, 4, 3, 2, 5],
           [1, 4, 3, 2, 5, 0, 0, 0],
           [1, 4, 3, 2, 4, 3, 2, 5]]),
    
    `pathNodeLengths`: TBD
    
    `pathDirArray`:
    array([[False, False, False, False, False, False, False, False],
           ...
           [False,  True,  True,  True, False, False, False, False],
           [False,  True,  True,  True,  True,  True,  True, False],
           [False,  True, False,  True, False, False, False, False],
           [False,  True, False,  True,  True, False,  True, False]]))
    
    `pathNodeLengthsCum`: TBD
    
    '''
    
    
    print('Preprocessing paths...')
    
    numPaths = len(graph.paths)
    numPathsZeros = np.int(np.ceil(np.log10(numPaths)))
    
    maxLengthPath = len(max(graph.paths,key=lambda arr: len(arr)))

    pathLengths = []
    pathNodeArray = np.zeros((len(graph.paths),maxLengthPath),dtype = np.int)
    pathNodeLengths = np.zeros((len(graph.paths),maxLengthPath),dtype = np.int)
    pathDirArray = np.zeros((len(graph.paths),maxLengthPath),dtype = np.bool) # True - "+", False - "-" or padding for shorter paths.

    for i,path in enumerate(graph.paths):
        print(f'\nProcessing path {i+1:0{numPathsZeros}}/{numPaths:0{numPathsZeros}}',end='')
        pathLengths.append(len(path))

        pathNodeArray[i,:pathLengths[-1]] = [int(node[:-1]) for node in path]
        pathDirArray[i,:pathLengths[-1]] = [node[-1]=='-' for node in path]
        pathNodeLengths[i,:pathLengths[-1]] = [nodeLengths[node-1] for node in pathNodeArray[i,:pathLengths[-1]]]

    pathNodeLengthsCum = np.cumsum(pathNodeLengths,axis=1)
    print('\nFinished preprocessing paths')
    return pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum

In [ ]:
#export
def getPathNodeInversionRate(pathNodeArray,pathDirArray,pathLengths,inversionThreshold=0.5):
    '''
    Generate a dict of dicts which stores information about inversion rate of each node for each path.
    
    Parameters
    ==========
    `pathLengths`: TBD (generated by `initialPathAnalysis`)

    `pathNodeArray`: numpy.array[int]. An array of the size number_of_paths x max_length_of_path, 
                    containing sequence of nodes in the order as they are passed by the given 
                    path. If the length of the path is less than the number of columns of the
                    array, then the unused columns are padded with zeros (node numbers are 1-based). 
                    Generated by `initialPathAnalysis`.
    
    `pathDirArray`: numpy.array[bool]. An array of the size number_of_paths x max_length_of_path, 
                    containing direction (normal or inversed) in which the given path passes each node. 
                    `False` means normal direction, `True` means inverted. If the length of the path 
                    is less than the number of columns of the array, then the unused columns 
                    are padded with `False`.
                    Generated by `initialPathAnalysis`.
    
    `inversionThreshold`: float (default: 0.5). A float from the range [0,1], which defines whether an inversion rate
                        is considered as normal or inverted in binary inversion. It is needed for link directions
                        and visualisation of the nodes. It is a dictionary with pathIDs as keys and dictionaries as values.
                        The dictionaries as values have nodeID as keys and boolean indicating whether this node for this path
                        is inverted (True) or not (False).
    
    '''
    
    pathNodeInversionRate = {}
    for pathID in range(pathNodeArray.shape[0]):
        nodeInversionRate = pathNodeInversionRate.setdefault(pathID,{})
        pathNodes = pathNodeArray[pathID,:pathLengths[pathID]]
        uniqueNodes = np.unique(pathNodes)
        for node in uniqueNodes:
            nodeInversionRate[node] = np.mean(pathDirArray[pathID,np.where(pathNodes==node)[0]])>inversionThreshold
    
    return pathNodeInversionRate

In [ ]:
#export
class LinkGetter:
    def __init__(self,nodes,links):
        self.links = links
        self.nodes = nodes
    
    def __getitem__(self,nodeID):
        try:
            node = self.links[nodeID]
            childrenList = list(itertools.chain(*list(node.values())))
            return list(set([el[0] for el in childrenList]))
        except KeyError:
            if nodeID>0 and nodeID<=len(self.nodes):
                return []
            else:
                raise KeyError(f'There is no node with ID {nodeID}')
    def __call__(self):
        allLinks = []
        for node in range(1,len(self.nodes)+1):
            res = self[node]
            if res is not None:
                allLinks.extend([(node,el) for el in res])
        return allLinks
#         return list(self)
    def __iter__(self):
        for i in range(len(self.nodes)):
            res = self[i+1]
            if res is not None:
                yield (i+1,res)
            else:
                yield (i+1,[])

In [ ]:
#export
class GenomeGraph:
    def __init__(self,gfaPath=None,doOverlapCleaning=True,
                 paths=None,
                 nodes=None,nodesData=None,links=None,
                 pathsDict=None,
                 sequenceFiles=None,annotationFiles=None,
                 doBack=False,**kwargs):
        
        '''
        Priority 1: If you pass gfaPath as actual path to gfa file, then it will be loaded ignoring other options
        accessionsToRemove: list or None (default). If not None, a list of strings, if any of the string contains 
        in pathname, the path will be ignored.
        In this case, the following options are available:
            `isGFASeq`: boolean (default: True). Whether the graph should be considered as sequence graph (True)
                        or as gene/block graph (False).
        
        Priority 4: If annotationFiles is not None, but is a list of paths to annotation (gff3) files, 
        then the following extra options are available:
            `fileOrder`: list or None (default) Order in which each accession should be loaded into the 
                         graph (and order in which paths will be represented)
            `doUS`: boolean (default: False) Add unrelated sequence blocks between annotated genes/blocks.
            `refAnnotationFile`: str. If given, it has to be a path to gff3 file with reference annotation 
                                 with AT notation for gene names. For this, `transMap` has to be provided.
            `refSequenceFile`: str or None (default). If provided with path, then it will be used to obtain 
                               sequences of each block/gene.
            `transMap`: a bidict object (`util` module) or None (default). A dictionary ontaining bidirectional 
                        relation between OG notation of genes and AT notation of genes. 
                        It is required if refAnnotationFile is provided.
            `seqSuffix`: str or None (default). Which sequence suffix within each annotation to use. 
                         If None, it will run all sequences in each annotation in lexicographic order
                         and join them together (e.g. end of chr1 will be joined to start of chr2).
            `refAccession`: str or None (default). Optional accession ID for reference annotation (if not the same as the file name).
        
        '''
        self.nodes = []
        self.nodesData = []
        self.nodesAnnotation = []
        self.nodesChr = []
        self.forwardLinks = {}
        self.overlaps = {}
        self.paths = []
        self.accessions = []
        self._nxGraph = None
        self.tremauxTree = None
        
        self.isBack = doBack
        
        if gfaPath is not None:
            self._loadGFA(gfaPath,isGFASeq=kwargs.get('isGFASeq',True),accessionsToRemove=kwargs.get('accessionsToRemove',None))
        elif nodes is not None and links is not None and paths is not None:
            self.nodes = nodes
            if nodesData is not None:
                self.nodesData = nodesData
            else:
                self.nodesData = ['']*len(self.nodes)
            self.forwardLinks = links
            self.paths = paths
        elif pathsDict is not None:
            self._graphFromPaths(pathsDict,**kwargs) # sequenceFiles can be None
        elif annotationFiles is not None:
            self._graphFromAnnotation(annotationFiles,sequenceFiles,**kwargs)
        
        self.order = list(range(1,len(self.nodes)+1))
        
        self.pureForwardlinks = LinkGetter(self.nodes,self.forwardLinks)

        self._pathCount()
        self.invertNodes()
        
        if doBack:
            self.backLinks = self._revertLinks()
            self.pureBackLinks = LinkGetter(self.nodes,self.backLinks)
        
        if len(self.overlaps)>0 and doOverlapCleaning:
            self.treeSort()
            pdb.set_trace()
            self.removeOverlaps()
            
    # check edgePaths calculation. Incorrectly calculates the numbers 
    # (one too many in one of group of incoming edges to each node)
    def _pathCount(self):
        # self.inPath = []
        # self.outPath = []
        self.nodePass = [0]*len(self.nodes)
        
        self.nodeStrandPaths = [[0,0] for _ in range(len(self.nodes))]
        self.pathStarts = [0]*len(self.nodes)
        self.inPath = [0]*len(self.nodes)
        self.outPath = [0]*len(self.nodes)
        self.edgePaths = {}
        
        self.nodePassUnique = {}
        self.inPathUnique = {}
        self.outPathUnique = {}
        self.edgePathsUnique = {}
        
        for pathID,path in enumerate(self.paths):
            
            previousNode = None
            for nodeStrand in path:
                pathNode = int(nodeStrand[:-1])-1
                pathStrand = int(nodeStrand[-1]=='-') # 0 if "+", 1 if "-"
                self.nodePass[pathNode] += 1
                self.nodePassUnique.setdefault(pathNode,set()).add(pathID)
                self.nodeStrandPaths[pathNode][pathStrand] += 1
                self.outPath[pathNode] += 1
                self.outPathUnique.setdefault(pathNode,set()).add(pathID)
                self.inPath[pathNode] += 1
                self.inPathUnique.setdefault(pathNode,set()).add(pathID)
                if previousNode is not None:
                    self.edgePaths.setdefault((previousNode+1,pathNode+1),0)
                    self.edgePaths[(previousNode+1,pathNode+1)] += 1
                    
                    self.edgePathsUnique.setdefault((previousNode+1,pathNode+1),set())\
                                            .add(pathID)
                else:
                    self.pathStarts[pathNode] += 1
                    self.inPath[pathNode] -= 1
                previousNode = pathNode
            self.outPath[previousNode] -= 1
        
        # We are not interested in pathIDs in *Unique blocks, so, 
        # converting them back to standard format.
        self.nodePassUnique = [len(self.nodePassUnique.get(key,[])) for key in range(len(self.nodes))]
        self.inPathUnique = [len(self.inPathUnique.get(key,[])) for key in range(len(self.nodes))]
        self.outPathUnique = [len(self.outPathUnique.get(key,[])) for key in range(len(self.nodes))]
        self.edgePathsUnique = {key:len(value) for key,value in self.edgePathsUnique.items()}
    
    def _revertLinks(self):
        backLinks = {}
        for fromNode,links in self.forwardLinks.items():
            for fromStrand,strandLinks in links.items():
                for item in strandLinks:
#                     if item[0] not in backLinks.keys():
#                         backLinks[item[0]] = {}
#                         backLinks[item[0]][item[1]] = []

#                     elif item[1] not in forwardLinks[item[0]].keys():
#                         backLinks[item[0]][item[1]] = []

                    backLinks.setdefault(item[0],{}).setdefault(item[1],[]).append((fromNode,fromStrand))
        return backLinks
    
    def _loadGFA(self, gfaFile, isGFASeq=True, accessionsToRemove=None):
        baseName = os.path.splitext(os.path.basename(gfaFile))[0]
        dirPath = os.path.dirname(gfaFile)
        jsonFile = f'{dirPath}{os.path.sep}nodeNames_{baseName}.json'
        annotationFile = f'{dirPath}{os.path.sep}annotation_{baseName}.dat'
        
        print(f'Loading graph from {gfaFile}')
        
        if os.path.exists(jsonFile):
            print(f'Found nodeNames file {jsonFile}, loading names.')
            nodeNames = json.load(open(jsonFile,mode='r'))
        else:
            nodeNames = None
        
        loadAnnotation = False
        
        if os.path.exists(annotationFile):
            print(f'Found node annotation file {annotationFile}, loading associations.')
            self.nodesAnnotation = joblib.load(annotationFile)
            emptyAnnotation = False
        else:
            emptyAnnotation = True
        
        prevPathSegment = None
        
        gfaList = open(gfaFile,mode='r').readlines()
        
        if gfaList[0][7:-1] == '2.0':
            raise NotImplementedError('At the moment only GFA v1 is supported')
        elif gfaList[0][7:-1] != '1.0':
            warnings.warn('Cannot identify version of GFA. Assuming it is GFA v1. If it is not, expect unpredictable results.',category=ResourceWarning)
       
        segmentList = [el for el in gfaList if el.lower().startswith('s')]
        linkList = [el for el in gfaList if el.lower().startswith('l')]
        pathStringsList = [el for el in gfaList if el.lower().startswith('p')]

        self.nodeNameToID = {}
        
        numSegments = len(segmentList)
        if numSegments>0:
            numSegmentDigits = int(np.ceil(np.log10(numSegments)))
        else:
            raise ValueError('Incorrect GFA file format. No segments were found')
        
        for nodeID,node in enumerate(segmentList):
            print(f'\nLoading segment {nodeID+1:0{numSegmentDigits}}/{numSegments:0{numSegmentDigits}}',end='')
            segmentArray = node.rstrip().split(sep='\t')
            segID,segGFAData = segmentArray[1:3]
            if isGFASeq:
                segSeq = segGFAData
            else:
                segSeq = ''
            
            if nodeNames is not None:
                segName = nodeNames[int(segID)-1]
            elif isGFASeq:
                segName = str(segID)
            else:
                segName = segGFAData
            
            self.nodeNameToID[segID] = len(self.nodes)+1
            
            self.nodes.append(segName)
            self.nodesData.append(segSeq)
        
        if emptyAnnotation:
            self.nodesAnnotation = [{}]*len(self.nodes)
            
        print('\nLoading segments finished.')
            
        numLinks = len(linkList)
        if numLinks>0:
            numLinkDigits = int(np.ceil(np.log10(numLinks)))
        else:
            numLinkDigits = 1
        
        for linkID,link in enumerate(linkList):
            print(f'\nLoading link {linkID+1:0{numLinkDigits}}/{numLinks:0{numLinkDigits}}',end='')
            linkArray = link.rstrip().split(sep='\t')
            fromNodeID,fromStrand,toNodeID,toStrand = linkArray[1:5]
            overlap = linkArray[5]
            fromNode = self.nodeNameToID[fromNodeID]
            toNode = self.nodeNameToID[toNodeID]
            if overlap[:-1].isnumeric() and overlap[-1]=='M':
                self.overlaps[(fromNode,fromStrand)] = int(overlap[:-1])
                
            curLink = self.forwardLinks.setdefault(fromNode,{})
            curStrand = curLink.setdefault(fromStrand,[])
            if (toNode,toStrand) not in curStrand:
                curStrand.append((toNode,toStrand))
        print('\nLoading links finished')
        
        numPaths = len(pathStringsList)
        if numPaths>0:
            numPathDigits = int(np.ceil(np.log10(numPaths)))
        else:
            raise ValueError('Incorrect GFA file format. No paths were found')
        
        addedPaths = 0
        ignoredPaths = 0
        for pathID,pathString in enumerate(pathStringsList):
            print(f'\nLoading path {pathID+1:0{numPathDigits}}/{numPaths:0{numPathDigits}}',end='')
            pathArray = pathString.rstrip().split(sep='\t')
            seqID,path = pathArray[1:3]
            useAccession = True
            if isinstance(accessionsToRemove,list):
                for accessionTemplate in accessionsToRemove:
                    if seqID.find(accessionTemplate)!=-1:
                        ignoredPaths += 1
                        useAccession = False
                        break
            if useAccession:
                self.paths.append([f'{self.nodeNameToID.get(ns[:-1],ns[:-1])}{ns[-1]}' for ns in path.split(',')])
                self.accessions.append(seqID)
                addedPaths += 1
        print(f'\nLoading paths finished. {addedPaths} paths added, {ignoredPaths} paths ignored.')
    
    def loadAnnotations(self, annotationPath,seqSuffix):
        if len(self.nodesData[0])==0:
            warnings.warn('Annotation can be applied only to sequence graph, but no sequence associated with nodes was found. Aborted.')
            return
        
        for accID,path in zip(self.accessions,self.paths):
            absPosStart = 0
            annotation = skbio_read(f'{annotationPath}{os.path.sep}{accID}.gff',format='gff3',into=IntervalMetadata,seq_id=f'{accID}{seqSuffix}')
            for nodestrand in path:
                nodeNum = int(nodesstrand[:-1])
                nodeDict = self.nodeAnnotation[nodeNum-1].setdefault(accID,{})
                nodeLen = len(self.nodesData[nodeNum-1])
                nodeAnnotationGen = annotation.query((absPosStart+1,absPosStart+nodeLen))
                for na in nodeAnnotationGen:
                    if na.bounds[0]<absPosStart+1:
                        leftNodeBound = 1
                    else:
                        leftNodeBound = na.bound[0] - absPosStart

                    if na.bound[1]<absPosStart+nodeLen:
                        rightNodeBound = na.bound[1] - absPosStart
                    else:
                        rightNodeBound = nodeLen
                        
                    ogName = na.metadata['OG']
                    atNamesStr = na.metadata.get('AT')
                    
                    nodeDict.setdefault(ogName,[]).append((leftNodeBound,rightNodeBound))
                    
                    if atNamesStr is not None:
                        atNameList = atNameStr.split(',')
                        for atName in atNameList:
                            nodeDict.setdefault(atName,[]).append((leftNodeBound,rightNodeBound))
    
    def updateAnnotationFromNodes(self,isSeq=True):
        '''
        This function is used only for primitive block graphs (e.g. gene and chain graphs)
        if there is no proper annotation available (e.g. graph was created from paths and 
        some extra information about nodes is needed).

        It takes "name" of each node either from `graph.nodes` (if `isSeq` is False) or
        from `graph.nodesData` (if `isSeq` is True).

        Parameters
        ##########

        `isSeq`: Whether it contains names as names or as seq.
        '''
        offset = 0
        for annID in range(len(self.nodesAnnotation)):
            if isSeq:
                nodeSeq = self.nodesData[annID]
            else:
                nodeSeq = self.nodes[annID]
            annEl = self.nodesAnnotation[annID]
            updatedAnnEl = {}
            for pathName in annEl:
                updatedAnnEl[pathName] = {nodeSeq:[(0,len(nodeSeq)-1)]}
            self.nodesAnnotation[annID] = updatedAnnEl
    
    def _graphFromNodesLinks(self,nodes,links):
        raise NotImplementedError('Generating graph from nodes and links is not yet implemented.')
    
    def _getNodeID(self,node,maxNodeNameLength,pathID):
        _node = node.zfill(maxNodeNameLength)
        try:
            nodeID = self.nodes.index(_node)+1
            self.nodesAnnotation[nodeID-1].setdefault(pathID,{}).setdefault(_node,[]).append((0,len(_node)-1))
        except ValueError:
            self.nodes.append(_node)
            self.nodesAnnotation.append({pathID:{_node:[(0,len(_node)-1)]}})
            nodeID = len(self.nodes)
            self.nodeNameToID[str(nodeID)] = nodeID
            
        return nodeID
    
    def _graphFromPaths(self,paths,sequenceFiles=None,nodeNameLength=None):
        if sequenceFiles is not None:
            warnings.warn("sequenceFiles reading with path is not yet implemented.")
        
        if not isinstance(paths,dict):
            raise ValueError(f"paths should be dict but {type(paths)} was given.")
        
#         if sequenceFiles is None:
        if nodeNameLength is None:
            maxNodeNameLength = len(max([max(path,key=lambda a: len(a)) for path in paths.values()],key=lambda a: len(a))) - 1
        else:
            maxNodeNameLength = nodeNameLength
        
        self.nodeNameToID = {}
        self.nodes = []
        self.paths = []
        self.accessions = []
        links = {}
        
        self.nodeNameToID = {}
        
        for pathID,path in paths.items():
            self.accessions.append(pathID)
            prevNode = path[0][:-1]
            prevDirection = path[0][-1]
            prevNodeID = self._getNodeID(prevNode,maxNodeNameLength,pathID)
            newPath = [f'{prevNodeID}{prevDirection}']
            for nodeDir in path[1:]:
                
                node = nodeDir[:-1]
                direction = nodeDir[-1]
                
                nodeID = self._getNodeID(node,maxNodeNameLength,pathID)
                newPath.append(f'{nodeID}{direction}')
                links.setdefault(prevNodeID,{}).setdefault(prevDirection,set()).add((nodeID,direction))
                
                prevNode = node
                prevDirection = direction
                prevNodeID = nodeID
            self.paths.append(newPath)
        
        
        self.forwardLinks = dict([(fromNode,dict([(fromStrand,list(toSet)) for fromStrand,toSet in strandDict.items()])) for fromNode,strandDict in links.items()])
        self.nodesData = ['']*len(self.nodes)
    
    def _processAnnotation(self, annotationFile, links, ATmap=None, seqFile=None, seqSuffix=None, doUS=False, isRef=False, accID=None):
        accessionID, genes, sequences = \
            processAccession(annotationFile,
                             seqFile,
                             ATmap=ATmap,
                             isRef=isRef,
                             accID=accID)
        if seqSuffix is None:
            seqList = genes.sequenceID.unique().tolist()
            seqList.sort()
        else:
            seqList = [f'{accessionID}{seqSuffix}']

        path = []
        for seqID in seqList:

            p, cigar, usCounter = generatePathsLinks(genes.loc[genes.sequenceID == seqID], seqID, accessionID, sequences, self.OGList,
                                                     self.nodes, self.nodesAnnotation, self.nodesChr, self.nodeNameToID, links,
                                                     self.usCounter, doUS=doUS, segmentData=self.nodesData)
            path = path + p
        if isRef:
            self.paths.insert(0,path)
            self.accessions.insert(0,accessionID)
        else:
            self.paths.append(path)
            self.accessions.append(accessionID)

        return links
    
    def _graphFromAnnotation(self,annotationFiles,sequenceFiles=None,**kwargs):
        self.nodeNameToID = {}

        fileOrder = kwargs.get('fileOrder',list(range(len(annotationFiles))))
        
        doUS = kwargs.get('doUS',False)
        self.usCounter = 0
        self.OGList = []
        links = self._linksDictToSet(self.forwardLinks)

        if 'refAnnotationFile' in kwargs:
            links = self._processAnnotation(kwargs['refAnnotationFile'], links,
                                            ATmap=kwargs.get('transMap', None),
                                            seqFile=kwargs.get('refSequenceFile',None),
                                            seqSuffix=kwargs.get('seqSuffix', None),
                                            doUS=doUS,
                                            isRef=True,
                                            accID=kwargs.get('refAccession',None))
        
        for fileNum in fileOrder:
            if sequenceFiles is not None:
                seqFile = sequenceFiles[fileNum]
            else:
                seqFile = None
             # annotationFile
             # transMap
            links = self._processAnnotation(annotationFiles[fileNum], links,
                                            ATmap=kwargs.get('transMap', None),
                                            seqFile=seqFile,
                                            seqSuffix=kwargs.get('seqSuffix', None),
                                            doUS=doUS)
        

        self.forwardLinks = self._linksSetToDict(links)
    
    def _linksDictToSet(self,dictLinks):
        setLinks = {}
        for fromNode,strandLinks in dictLinks.items():
            for fromStrand,linkList in strandLinks.items():
                for toNode,toStrand in linkList:
                    fromNodeStrand = f'{fromNode}{fromStrand}'
                    toNodeStrand = f'{toNode}{toStrand}'
                    if fromNodeStrand not in setLinks.keys():
                        setLinks[fromNodeStrand] = set()
                    setLinks[fromNodeStrand].add(toNodeStrand)
        return setLinks
    
    def _linksSetToDict(self,setLinks):
        dictLinks = {}
        for fromNodeStrand,links in setLinks.items():
            fromNode = int(fromNodeStrand[:-2])
            fromStrand = fromNodeStrand[-1]
            for toNodeStrand in links:
                toNode = int(toNodeStrand[:-2])
                toStrand = toNodeStrand[-1]
                
                if fromNode not in dictLinks.keys():
                    dictLinks[fromNode] = {}
                    dictLinks[fromNode][fromStrand] = []

                elif fromStrand not in dictLinks[fromNode].keys():
                    dictLinks[fromNode][fromStrand] = []

                dictLinks[fromNode][fromStrand].append((toNode,toStrand))
        return dictLinks
    
    def generateTremauxTree(self,byPath=True):
        _nxGraph = nx.DiGraph()
        
        for nodeID,toList in self.pureForwardlinks:
            if toList is not None:
                edgeList = [(nodeID,toItem) for toItem in toList]
                _nxGraph.add_edges_from(edgeList)
            
        self.tremauxTree = TremauxTree(_nxGraph,self,byPath)
        
    def addAccessionAnnotation(self,annotationFile,sequenceFile=None):
        '''
        Ideally, a function should be able to add one accesstion to existing graph. 
        When implemented, `_graphFromAnnotation` should be using this function.
        
        '''
        pass
    
    def addLink(self,fromNode,fromStrand,toNode,toStrand):
        # Need testing
        if fromNode not in self.nodes:
            raise IndexError(f'Node {fromNode} is not in nodes list')
        if toNode not in self.nodes:
            raise IndexError(f'Node {toNode} is not in nodes list')
        
        if fromNode not in self.forwardLinks.keys():
            self.forwardLinks[fromNode] = {}
            self.forwardLinks[fromNode][fromStrand] = []
            
        elif fromStrand not in self.forwardLinks[fromNode].keys():
            self.forwardLinks[fromNode][fromStrand] = []

        self.forwardLinks[fromNode][fromStrand].append([toNode,toStrand])

        if self.isBack:
            if toNode not in self.backLinks.keys():
                self.backLinks[toNode] = {}
                self.backLinks[toNode][toStrand] = []

            elif toStrand not in self.backLinks[toNode].keys():
                self.backLinks[toNode][toStrand] = []

            self.backLinks[toNode][toStrand].append([fromNode,fromStrand])
    
    def addNode(self,nodeID,data=None):
        # Need testing
        if nodeID not in self.nodes:
            self.nodes.append(nodeID)
            self.nodesData[nodeID] = data
        else:
            warnings.warn(f'You attempted to add node with {nodeID}, but it already exists in the graph. The addition was ignored.')
        pass
    
    def _getStartNode(self,bubbleNode):
#         allPaths = list(nx.shortest_path(self.tremauxTree,None,bubbleNode).values())
#         startList = []
#         for path in allPaths:
#             if len(startList)>0:
#                 for node in startList.copy():
#                     if nx.has_path(self.tremauxTree,node,path[0]):
#                         continue
#                     if nx.has_path(self.tremauxTree,path[0],node):
#                         startList.remove(node)

#             startList.append(path[0])

#         return startList
        rootNodes = self.tremauxTree.getRootNodes()
        
        for root in rootNodes:
            if nx.has_path(self.tremauxTree,root,bubbleNode):
                return root
                
        return None
    
    def _getEdgeValue(self,start,end,unique=False):
        
        if start is None:
            endInPathRatio = (self.nodePass[end-1]-self.inPath[end-1])/self.nodePass[end-1]
            if unique:
                return self.nodePassUnique[end-1]*endInPathRatio
            else:
                return self.nodePass[end-1]*endInPathRatio
        else:
            startOutPathRatio = self.edgePaths.get((start,end),0)/self.outPath[start-1]
            endInPathRatio = self.edgePaths.get((start,end),0)/self.inPath[end-1]
            if unique:
                return self.edgePathsUnique.get((start,end),0)*startOutPathRatio*endInPathRatio
            else:
                return self.edgePaths.get((start,end),0)*startOutPathRatio*endInPathRatio
    
    def treeSort(self,byPath=True,bubblePriorityThreshold=0.5):
        
        print('Constructing Tremaux tree')
        self.generateTremauxTree(byPath)
        print('Done!')
        queue = []
        stopNodes = set()
        stopNodesOrigin = {}
        processed = []
        self.order = []

#         pdb.set_trace()
        
        print('Getting root nodes')
        rootNodes = self.tremauxTree.getRootNodes()
#         self.order.append(rootNode)
        
        # The first root node won't be added to the order automatically,
        # but will be processed as all other nodes.
        startNode = rootNodes.pop()
#         self.order.append(startNode)
#         descendantsToAdd = list(self.tremauxTree.edges(startNode))
#         descendantsToAdd.sort(key=lambda edge: self.edgePathsUnique.get(edge,0))
#         queue.extend(descendantsToAdd)
        queue.append((None,startNode))
        
        print('Start Loop...')
        while len(queue)>0 or len(rootNodes)>0:
            if len(queue)==0:
                try:
                    while len(queue)==0:
                        startNode = rootNodes.pop()
#                         descendantsToAdd = list(self.tremauxTree.edges(startNode))
#                         if len(descendantsToAdd)>0:
#                             descendantsToAdd.sort(key=lambda edge: self.edgePathsUnique.get(edge,0))
#                             queue.extend([edge for edge in descendantsToAdd if edge not in processed])
#                         if startNode not in self.order:
#                             self.order.append(startNode)
                        queue.append((None,startNode))
                except IndexError:
                    break
            
            startNode,endNode = queue.pop()
            
#             if endNode==840:
#                 pdb.set_trace()
            
#             print(f'Queue: {len(queue)} - Processed: {len(processed)} - Order: {len(self.order)} - Start: {startNode} - End: {endNode}')
            print(f'\nNodes in order: {len(self.order)}/{len(self.nodes)}',end='')
    
            if (startNode,endNode) in processed:
                continue
            if (startNode in self.order and endNode in self.order):
                if not ((startNode,endNode) in processed):
                    processed.append((startNode,endNode))
                continue
            
            endNodeAdded = False
            
            bubblePriorityQueue = []
            bubbleEdges = self.tremauxTree.bubbleEdges.inverse.get(endNode,[])+[startNode]
            if len(bubbleEdges)>0:
                if byPath:
                    bubbleEdgesOrder = np.argsort([self._getEdgeValue(edge,endNode) for edge in bubbleEdges])
                else:
                    bubbleEdgesOrder = list(range(len(bubbleEdges)))
                # Chech that bubble node is not in order already
                # Check edgePaths for all outgoing links (including non-bubbles but not loops), use this bubble 
                # only if the bubble edge has maximum number of paths.
                # if this bubble should be used compare edgePaths for bubble edge and for normal edge to 
                # current node. If normal edge is larger, then add current node to order and node past LCA 
                # on alternative path(s) to the queue (to be processed next) and after them the descendandts of 
                # current node. Otherwise, do not add current node to order, but add nodes past LCA and then (to processed later) current node to queue.
                
                for bEdge in bubbleEdgesOrder:
                    bubbleNode = bubbleEdges[bEdge]
                    if bubbleNode in self.order and bubbleNode!=startNode:
                        continue
                    if byPath:
                        curBubbleEdgePaths = self._getEdgeValue(bubbleNode,endNode)
                        if bubbleNode is not None:
                            bubbleNodeOutEdges = list(self.tremauxTree.out_edges(bubbleNode)) + \
                                                [(bubbleNode,node) for node in self.tremauxTree.bubbleEdges.get(bubbleNode,[])]
                            bubbleNodeOutEdgesValue = [self._getEdgeValue(*edge) for edge in bubbleNodeOutEdges]
                        else:
                            bubbleNodeOutEdgesValue = [self._getEdgeValue(bubbleNode,endNode)]
                        
                        if max(bubbleNodeOutEdgesValue)>curBubbleEdgePaths:
                            continue
                        
#                         if bubbleNode is not None:
#                             if self._getEdgeValue(bubbleNode,endNode,unique=False)/self.inPath[endNode-1]>bubblePriorityThreshold or \
#                             self._getEdgeValue(bubbleNode,endNode,unique=False)/self.outPath[bubbleNode-1]>bubblePriorityThreshold:
#                                 bubblePriority = True
#                             else:
#                                 bubblePriority = False
#                         else:
#                             bubblePriority = True
                        bubblePriority = True
    
                    if bubbleNode!=startNode:
                        # We are not on the main (original) edge
                        lca = nx.lowest_common_ancestor(self.tremauxTree,endNode,bubbleNode)
                        if byPath:
                            if lca is None:
                                _lca = self._getStartNode(bubbleNode)
    #                             startBubbleNodes.sort(key=lambda node: self.nodePassUnique[node-1])
    #                             lcaDescendantsToProcess = []
    #                             for startBubbleNode in startBubbleNodes: 
    #                                 snDescendantsToProcess = [(startBubbleNode,node) for node in list(self.tremauxTree[startBubbleNode])\
    #                                                           if node not in self.order and nx.has_path(self.tremauxTree,node,bubbleNode)]
    #                                 snDescendantsToProcess.sort(key=lambda node: self.edgePathsUnique.get((startBubbleNode,node),0))
    #                                 lcaDescendantsToProcess.extend(snDescendantsToProcess)

                            else: 
                                _lca = lca
                #                             lcaDescendantsToProcess = [(lca,node) for node in list(self.tremauxTree[lca]) \
    #                                                        if node not in self.order and nx.has_path(self.tremauxTree,node,bubbleNode)]
    #                             lcaDescendantsToProcess.sort(key=lambda node: self.edgePathsUnique.get((lca,node),0))
                            if _lca==bubbleNode:
                                # BubbleNode is the top of another tree
                                lcaDescendantsToProcess = [(None,bubbleNode)]
                            else:
                                lcaDescendantsToProcess = [(_lca,node) for node in list(self.tremauxTree[_lca])\
                                                                  if node not in self.order and nx.has_path(self.tremauxTree,node,bubbleNode)]
                                lcaDescendantsToProcess.sort(key=lambda edge: self.edgePathsUnique.get(edge,0))

                            for edge in lcaDescendantsToProcess:
                                if edge in queue:
                                    queue.remove(edge)
                            if len(lcaDescendantsToProcess)>0:
                                if bubblePriority:
                                    bubblePriorityQueue.append((bubbleNode,[edge for edge in lcaDescendantsToProcess if edge not in processed]))
                                else:
                                    queue.extend([edge for edge in lcaDescendantsToProcess if edge not in processed and edge[0] is not None])
                        else:
                            # TODO!!!
                            # rewrite pure graph sorting to process alternative branches by adding the top nodes 
                            # of each branch to the queue for immediate processing.
                            pathToAdd = nx.shortest_path(self.tremauxTree,lca,bubbleNode)
                            if lca is None:
                                allPaths = list(pathToAdd.values())
                                allPaths.sort(key=len)
                                pathToAdd = allPaths[-1]
                            else:
                                if endNode not in self.order and byPath:
                                    self.order.append(endNode)
                                    endNodeAdded = True
                            # WRONG!
                            self.order.extend([node for node in pathToAdd if node not in self.order])
                    else:
                        # We consider startNode as bubble
                        if byPath:
                            if bubblePriority:
                                bubblePriorityQueue.append((bubbleNode,(startNode,endNode)))
                            else:
                                # Do we need to add the current edge back to queue???
                                pass
                            
                            
            bubblePriorityQueue.sort(key=lambda b: self._getEdgeValue(b[0],endNode))
            if len(bubblePriorityQueue)>=1:
#             or \
#                 (len(bubblePriorityQueue)==1 and bubblePriorityQueue[-1][1]!=(startNode,endNode)):
#                 queue.append((startNode,endNode))#Why would I add the processed block again? Should I add descendant edges of the end node here? 
                # It should save a lot of wasted time and speed up the process.
                queueLen = len(queue)
                curEdgePassed = False
                while len(bubblePriorityQueue)>0:
                    bNode,bToAdd = bubblePriorityQueue.pop()
                    
                    if bNode!=startNode:
                        queue.extend(bToAdd)
                        stopNodes.add(bNode)
                        stopNodesOrigin[bNode] = endNode
                    else:
                        if len(bubblePriorityQueue)==0:
                            endNodeAdded = self._addNextEdgesToQueue(startNode,endNode,
                                                    processed,queue,stopNodes,stopNodesOrigin)
                            curEdgePassed = True
                        else:
                            queue.append((startNode,endNode))
                if not curEdgePassed:
                    queue[queueLen:queueLen] = [(startNode,endNode)]
            else:
                endNodeAdded = self._addNextEdgesToQueue(startNode,endNode,
                                                    processed,queue,stopNodes,stopNodesOrigin)
        print()
    
    def _addNextEdgesToQueue(self,startNode,endNode,processed,queue,stopNodes,stopNodesOrigin):
        endNodeAdded = False
        processed.append((startNode,endNode))
        # Should we add start or end???
        if endNode not in self.order:
            self.order.append(endNode)
            endNodeAdded = True

        edgesToAdd = [edge for edge in list(self.tremauxTree.edges(endNode)) if edge not in processed and edge not in queue]

        edgesToAdd.sort(key=lambda edge: self._getEdgeValue(*edge))
        if endNode in stopNodes and len(edgesToAdd)>0:
            bubbleEnd = stopNodesOrigin[endNode]
            bubbleEndEdge = list(self.tremauxTree.in_edges(bubbleEnd))[0]
            try: 
                bubbleEndEdgeInd = queue.index(bubbleEndEdge)
            except ValueError:
                bubbleEndEdgeInd = len(queue)
            
            queue[bubbleEndEdgeInd:bubbleEndEdgeInd] = edgesToAdd
#             queue.append(bubbleEndEdge)
            stopNodes.remove(endNode)
            del stopNodesOrigin[endNode]
        else:
            queue.extend(edgesToAdd)
        
        return endNodeAdded
    
    def toGFA(self,gfaFile,doSeq=True):
        '''
            Recording existing graph structures to GFA v1 file.
        '''
        baseName = os.path.splitext(os.path.basename(gfaFile))[0]
        dirPath = os.path.dirname(gfaFile)
        jsonFile = f'{dirPath}{os.path.sep}nodeNames_{baseName}.json'
        annotationFile = f'{dirPath}{os.path.sep}annotation_{baseName}.dat'
        
        gfaWriter = open(gfaFile,mode='w')
        
        gfaWriter.write('H\tVN:Z:1.0\n')
        
        translator = {}
        nodeNameList = []
        nodesAnnotation = []
        
        for i,nodeID in enumerate(self.order):
            nodeSeq = self.nodesData[nodeID-1]
            nodeName = self.nodes[nodeID-1]
            nodesAnnotation.append(self.nodesAnnotation[nodeID-1])
            if doSeq and len(nodeSeq)>0:
                gfaWriter.write(f'S\t{i+1}\t{nodeSeq}\n')
                nodeNameList.append(nodeName)
            else:
                gfaWriter.write(f'S\t{i+1}\t{nodeName}\n')
            
            translator[nodeID] = i+1
        
#         isInversionDict = self._generateIsInversionDict()
#         strandInversionDict = {'+':'-','-':'+'}
        
        for fromNode,strandLinks in self.forwardLinks.items():
            for fromStrand,toLinkStrands in strandLinks.items():
                for toNode,toStrand in toLinkStrands:
                    
#                     if isInversionDict.get(fromNode,False):
#                         _fromStrand = strandInversionDict[fromStrand]
#                     else:
#                         _fromStrand = fromStrand

#                     if isInversionDict.get(toNode,False):
#                         _toStrand = strandInversionDict[toStrand]
#                     else:
#                         _toStrand = toStrand
                    gfaWriter.write(f'L\t{translator[fromNode]}\t{fromStrand}\t{translator[toNode]}\t{toStrand}\t*\n')
        
        for path,accessionID in zip(self.paths,self.accessions):
            newPath = []
            for nodeStrand in path:
                node = int(nodeStrand[:-1])#self.nodeNameToID.get(nodeStrand[:-1],nodeStrand[:-1])
#                 if isInversionDict.get(node,False):
#                     strand = strandInversionDict[nodeStrand[-1]]
#                 else:
                strand = nodeStrand[-1]
                newPath.append(f'{translator[node]}{strand}')
            gfaWriter.write(f'P\t{accessionID}\t{",".join(newPath)}\t*\n')
                
        gfaWriter.write('\n')
        gfaWriter.close()
        
        if len(nodeNameList)==len(self.nodes):
            with open(jsonFile,'w') as jsf:
                json.dump(nodeNameList,jsf)
                
        if len(nodesAnnotation)==len(self.nodes):
            joblib.dump(nodesAnnotation,annotationFile)
    
    # Node inversion functionality
    def invertNodes(self):
        nodeLengths = calcNodeLengths(self)
        pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(self,nodeLengths)

        for nodeID in range(len(self.nodes)):
            if self.nodeStrandPaths[nodeID][1]>self.nodeStrandPaths[nodeID][0]:
                print(f'\nNode {nodeID+1} inverted',end='')
                self._invertNode(nodeID,pathNodeArray)

        self._pathCount()
        print()

    def _invertNode(self,nodeID,pathNodeArray):
        
        self.nodesData[nodeID] = inverseSequence(self.nodesData[nodeID])
        
        pathIDs,positions = np.where(pathNodeArray==nodeID+1)
        strandreversal = {'+':'-','-':'+'}
        for pathID,pos in zip(pathIDs,positions):
            nodeStrand = self.paths[pathID][pos]
            self.paths[pathID][pos] = nodeStrand[:-1]+strandreversal[nodeStrand[-1]]

        nodeFromLink = self.forwardLinks.get(nodeID+1,{})
        positiveStrand = nodeFromLink.get('+',[])
        negativeStrand = nodeFromLink.get('-',[])

        if len(positiveStrand)>0:
            self.forwardLinks[nodeID+1]['-'] = positiveStrand
        else:
            if len(negativeStrand)>0:
                del self.forwardLinks[nodeID+1]['-']

        if len(negativeStrand)>0:
            self.forwardLinks[nodeID+1]['+'] = negativeStrand
        else:
            if len(positiveStrand)>0:
                del self.forwardLinks[nodeID+1]['+']

        revertLinks = self._revertLinks()        
        nodeToLink = revertLinks.get(nodeID+1,{})
        positiveStrand = nodeToLink.get('+',[])
        negativeStrand = nodeToLink.get('-',[])

        for fromNode,fromStrand in positiveStrand:
            self.forwardLinks[fromNode][fromStrand].remove((nodeID+1,'+'))
            self.forwardLinks[fromNode][fromStrand].append((nodeID+1,'-'))

        for fromNode,fromStrand in negativeStrand:
            self.forwardLinks[fromNode][fromStrand].remove((nodeID+1,'-'))
            self.forwardLinks[fromNode][fromStrand].append((nodeID+1,'+'))
            
        positiveOverlap = self.overlaps.pop((nodeID+1,'+'),0)
        negativeOverlap = self.overlaps.pop((nodeID+1,'-'),0)
        if positiveOverlap>0:
            self.overlaps[(nodeID+1,'-')] = positiveOverlap
        if negativeOverlap>0:
            self.overlaps[(nodeID+1,'+')] = negativeOverlap
    
    # End of node inversion functionality
    
    # node removal functionality
    def _removePositionsFromPaths(self,pathIDs,positions):
        print('Removing empty nodes from paths')
        print('paths')
        print(self.paths)
        for pathID in np.unique(pathIDs):
            offset = 0
            posArray  = np.sort(np.array(positions)[np.array(pathIDs)==pathID])
            for pos in posArray:
                print(f'{pathID} - {pos}')
                del self.paths[pathID][pos-offset]
                offset += 1
    
    def _updateLinkList(self,strandList,offsetDict):
        newStrand = []
        for toNode,toStrand in strandList:
            newStrand.append((offsetDict[toNode-1]+1,toStrand))
        return newStrand
    
    def _clearNodes(self,nodeIDs):
        print('nodeIDs')
        print(nodeIDs)
        print('paths')
        print(self.paths)
        offsetDict = {node:node for node in range(len(self.nodes))}
        for nodeID in nodeIDs:
            print(f'Removing node {nodeID+1}')
            updatedNodeID = offsetDict[nodeID]
            nodeName = self.nodes[updatedNodeID]
            del self.nodeNameToID[nodeName]
            del self.nodes[updatedNodeID]
            del self.nodesData[updatedNodeID]
            self.order.remove(nodeID+1)
            del offsetDict[nodeID]
            offsetDict.update({k:v-1 for k,v in offsetDict.items() if k>nodeID})
            
        
        self.order = [offsetDict[el-1]+1 for el in self.order]
        self.nodeNameToID.update({k:offsetDict[v-1]+1 for k,v in self.nodeNameToID.items()})
        
        for pathID in range(len(self.paths)):
            self.paths[pathID] = [f'{offsetDict[int(el[:-1])-1]+1}{el[-1]}' for el in self.paths[pathID]]
        
        print('paths')
        print(self.paths)
        print('offsetDict')
        print(offsetDict)
        
        nodesInForwardLinks = list(self.forwardLinks.keys())
        for node in sorted(nodesInForwardLinks):
            nodeDict = self.forwardLinks[node]
            newNode = offsetDict[node-1]+1
            if node!=newNode:
                del self.forwardLinks[node]
            forwardStrand = self._updateLinkList(nodeDict.get('+',[]),offsetDict)
            inverseStrand = self._updateLinkList(nodeDict.get('-',[]),offsetDict)

            if len(forwardStrand)>0:
                self.forwardLinks.setdefault(newNode,{})['+'] = forwardStrand
            if len(inverseStrand)>0:
                self.forwardLinks.setdefault(newNode,{})['-'] = inverseStrand
    
    def _removeNode(self,nodeID,revertLinks,pathNodeArray):
        pathIDs,positions = np.where(pathNodeArray==nodeID+1)

        excludeTuples = [(nodeID+1,'+'),(nodeID+1,'-')]

        nodeToLink = revertLinks.get(nodeID+1,{})
        toPositiveStrand = nodeToLink.get('+',[])
        toNegativeStrand = nodeToLink.get('-',[])

        nodeFromLink = self.forwardLinks.get(nodeID+1,{})
        fromPositiveStrand = nodeFromLink.get('+',[])
        fromNegativeStrand = nodeFromLink.get('-',[])

        for fromNode,fromStrand in toPositiveStrand:
            # Amending forwardLinks for positive strand of current node
            forwardNodeStrand = self.forwardLinks.get(fromNode,{}).get(fromStrand,[])
            forwardNodeStrand = list(set(forwardNodeStrand).union(fromPositiveStrand).difference(excludeTuples))
            self.forwardLinks[fromNode][fromStrand] = forwardNodeStrand

        for fromNode,fromStrand in toNegativeStrand:
            # Amend forwardLinks for negative strand of current node
            forwardNodeStrand = self.forwardLinks.get(fromNode,{}).get(fromStrand,[])
            forwardNodeStrand = list(set(forwardNodeStrand).union(fromNegativeStrand).difference(excludeTuples))
            self.forwardLinks[fromNode][fromStrand] = forwardNodeStrand

        for toNode,toStrand in fromPositiveStrand:
            # Amending revertLinks for positive strand of current node
            revertNodeStrand = revertLinks.get(toNode,{}).get(toStrand,[])
            revertNodeStrand = list(set(revertNodeStrand).union(toPositiveStrand).difference(excludeTuples))
            revertLinks[toNode][toStrand] = revertNodeStrand

        for toNode,toStrand in fromNegativeStrand:
            # Amend revertLinks for negative strand of current node
            revertNodeStrand = revertLinks.get(toNode,{}).get(toStrand,[])
            revertNodeStrand = list(set(revertNodeStrand).union(toNegativeStrand).difference(excludeTuples))
            revertLinks[toNode][toStrand] = revertNodeStrand

        del revertLinks[nodeID+1]
        del self.forwardLinks[nodeID+1]

        return pathIDs,positions
    
    def removeNodes(self,nodeIDsToRemove):
        revertLinks = self._revertLinks()
        nodeLengths = calcNodeLengths(self)
        _,pathNodeArray,_,_,_ = initialPathAnalysis(self,nodeLengths)

        pathIDs = []
        positions = []
        for nodeID in nodeIDsToRemove:
            pID,pos = self._removeNode(nodeID,revertLinks,pathNodeArray)
            pathIDs.extend(pID)
            positions.extend(pos)

        self._removePositionsFromPaths(pathIDs,positions)
        self._clearNodes(nodeIDsToRemove)

        self._pathCount()
    
    #end of node removal functionality
    
    # Node substitution functionality

    # End of node substitution functionality
    
    # Overlap removal functionality
    def _linkBounce(self,fromNodeStart,fromStrandStart,revertLinks,nodeLengths,kmerOverlap,cutOffsetRight,leftCut,rightCut):
        cutOffsetLeft = 0
        leftToCut = [(fromNodeStart,fromStrandStart)]
        rightToCut = []
        leftToProcess = [(fromNodeStart,fromStrandStart)]
        rightToProcess = []
    #     pdb.set_trace()
        while (len(leftToProcess)>0 or len(rightToProcess)>0):
            for node,strand in leftToProcess:
                rightSide = self.forwardLinks.get(node,{}).get(strand,[])
                for toNode,toStrand in rightSide:
                    if toStrand=='+':
                        cutOffsetLeft = max(leftCut.get(toNode,0),cutOffsetLeft)
                    else:
                        cutOffsetLeft = max(rightCut.get(toNode,0),cutOffsetLeft)

                    if cutOffsetLeft<kmerOverlap-cutOffsetRight and (toNode,toStrand) not in rightToCut:# and nodeLengths[toNode-1]>0:
                        rightToCut.append((toNode,toStrand))
                        rightToProcess.append((toNode,toStrand))
            leftToProcess = []

            for node,strand in rightToProcess:
                leftSide = revertLinks.get(node,{}).get(strand,[])
                for fromNode,fromStrand in leftSide:
                    if fromStrand=='+':
                        cutOffsetRight = max(rightCut.get(fromNode,0),cutOffsetRight)
                    else:
                        cutOffsetRight = max(leftCut.get(fromNode,0),cutOffsetRight)

                    if cutOffsetRight<kmerOverlap-cutOffsetLeft and (fromNode,fromStrand) not in leftToCut:# and nodeLengths[fromNode-1]>0:
                        leftToCut.append((fromNode,fromStrand))
                        leftToProcess.append((fromNode,fromStrand))
            rightToProcess = []

        return leftToCut,rightToCut,cutOffsetLeft,cutOffsetRight
    
    def _processBouncedLink(self,leftToCut,rightToCut,kmerOverlap,cutOffsetLeft,cutOffsetRight,nodeLengths,leftCut,rightCut,leftForbidden,rightForbidden):
        toCut = kmerOverlap - cutOffsetLeft - cutOffsetRight
        minLengthRight = min([nodeLengths[node-1] for node,strand in rightToCut])
        minLengthLeft = min([nodeLengths[node-1] for node,strand in leftToCut])
        isRightForbidden = np.any([ns in rightForbidden for ns in rightToCut ])
        isLeftForbidden = np.any([ns in leftForbidden for ns in leftToCut ])
        
        if isRightForbidden and isLeftForbidden:
            return
        
        if (minLengthRight<minLengthLeft and not isRightForbidden) or isLeftForbidden:
            sideToCut = rightToCut
            sideToKeep = leftToCut
            globalPositiveSide = leftCut
            globalNegativeSide = rightCut
            side = 'right'
        else:
            sideToCut = leftToCut
            sideToKeep = rightToCut
            globalPositiveSide = rightCut
            globalNegativeSide = leftCut
            side = 'left'

        maxCutAdjustment = max([max(toCut - nodeLengths[node-1],0) for node,strand in sideToCut])
        if toCut>maxCutAdjustment:
            for node,strand in sideToCut:
                if strand=='+':
                    if side =='right':
                        self.nodesData[node-1] = self.nodesData[node-1][toCut-maxCutAdjustment:]
                    else:
                        self.nodesData[node-1] = self.nodesData[node-1][:-(toCut-maxCutAdjustment)]
                    globalPositiveSide[node] = globalPositiveSide.get(node,0) + toCut - maxCutAdjustment
                else:
                    if side =='left':
                        self.nodesData[node-1] = self.nodesData[node-1][toCut-maxCutAdjustment:]
                    else:
                        self.nodesData[node-1] = self.nodesData[node-1][:-(toCut-maxCutAdjustment)]
                    globalNegativeSide[node] = globalNegativeSide.get(node,0) + toCut - maxCutAdjustment
                nodeLengths[node-1] -= toCut - maxCutAdjustment

        if maxCutAdjustment>0:
            # adjust the other side as well.
            for node,strand in sideToKeep:
                if strand=='+':
                    if side =='right':
                        self.nodesData[node-1] = self.nodesData[node-1][:-(maxCutAdjustment)]
                    else:
                        self.nodesData[node-1] = self.nodesData[node-1][maxCutAdjustment:]
                    globalNegativeSide[node] = globalNegativeSide.get(node,0) + maxCutAdjustment
                else:
                    if side =='left':
                        self.nodesData[node-1] = self.nodesData[node-1][:-(maxCutAdjustment)]
                    else:
                        self.nodesData[node-1] = self.nodesData[node-1][maxCutAdjustment:]
                    globalPositiveSide[node] = globalPositiveSide.get(node,0) + maxCutAdjustment
                nodeLengths[node-1] -= maxCutAdjustment
        for nodeStrand in leftToCut:
            del self.overlaps[nodeStrand]

    def removeOverlaps(self):
        nodeIDsToRemove = []
        leftCut = {}
        rightCut = {}

        leftForbidden = list(set([(int(path[-1][:-1]),path[-1][-1]) for path in self.paths])) # Nodes which should not be cut on the left side of link
        rightForbidden = list(set([(int(path[0][:-1]),path[-1][-1]) for path in self.paths])) # Nodes which should not be cut on the right side of link

        revertLinks = self._revertLinks()
        nodeLengths = calcNodeLengths(self)
        print('nodeLengths')
        print(nodeLengths)

        for fromNode in self.order:
            print('-----------')
            print(f'Processing node {fromNode}')
            if (fromNode,'+') in self.overlaps:
                print(f'Working on forward strand')
                kmerOverlap = self.overlaps[(fromNode,'+')]
                cutOffsetRight = rightCut.get(fromNode,0)
                print(f'Initial overlap={kmerOverlap}')
                print(f'Initial cutOffsetRight={cutOffsetRight}')
                if cutOffsetRight<kmerOverlap and nodeLengths[fromNode-1]>0:
                    leftToCut,rightToCut,cutOffsetLeft,cutOffsetRight = self._linkBounce(fromNode,'+',
                                                                                         revertLinks,
                                                                                         nodeLengths,
                                                                                         kmerOverlap,
                                                                                         cutOffsetRight,
                                                                                         leftCut,
                                                                                         rightCut)
                    print('leftToCut')
                    print(leftToCut)
                    print('rightToCut')
                    print(rightToCut)
                    print(f'cutOffsetLeft={cutOffsetLeft}; cutOffsetRight={cutOffsetRight}')
                    # process only if both leftToCut and rightToCut have some elements
                    if len(leftToCut)>0 and len(rightToCut)>0:
                        self._processBouncedLink(leftToCut,
                                                 rightToCut,
                                                 kmerOverlap,
                                                 cutOffsetLeft,
                                                 cutOffsetRight,
                                                 nodeLengths,
                                                 leftCut,
                                                 rightCut,
                                                 leftForbidden,
                                                 rightForbidden)
                    print('leftCut')
                    print(leftCut)
                    print('rightCut')
                    print(rightCut)
                    print('nodeLengths')
                    print(nodeLengths)

            if (fromNode,'-') in self.overlaps:
                print(f'Working on inverse strand')
                kmerOverlap = self.overlaps[(fromNode,'-')]
                cutOffsetRight = leftCut.get(fromNode,0)
                print(f'Initial overlap={kmerOverlap}')
                print(f'Initial cutOffsetRight={cutOffsetRight}')
                if cutOffsetRight<kmerOverlap and nodeLengths[fromNode-1]>0:
                    leftToCut,rightToCut,cutOffsetLeft,cutOffsetRight = self._linkBounce(fromNode,'-',
                                                                                         revertLinks,
                                                                                         nodeLengths,
                                                                                         kmerOverlap,
                                                                                         cutOffsetRight,
                                                                                         leftCut,
                                                                                         rightCut)
                    print('leftToCut')
                    print(leftToCut)
                    print('rightToCut')
                    print(rightToCut)
                    print(f'cutOffsetLeft={cutOffsetLeft}; cutOffsetRight={cutOffsetRight}')
                    if len(leftToCut)>0 and len(rightToCut)>0:
                        self._processBouncedLink(leftToCut,
                                                 rightToCut,
                                                 kmerOverlap,
                                                 cutOffsetLeft,
                                                 cutOffsetRight,
                                                 nodeLengths,
                                                 leftCut,
                                                 rightCut,
                                                 leftForbidden,
                                                 rightForbidden)
                    print('leftCut')
                    print(leftCut)
                    print('rightCut')
                    print(rightCut)
                    print('nodeLengths')
                    print(nodeLengths)
        nodeIDsToRemove = [i for i,l in enumerate(nodeLengths) if l==0]
        if len(nodeIDsToRemove)>0:
            self.removeNodes(nodeIDsToRemove)
    # End of overlap removal functionality
    
    def __iter__(self,forward=True):
        print(forward)
        for i in range(len(self.nodes)):
            yield i+1,self.forwardLinks[i+1]
    
    def __getitem__(self,ind):
        if isinstance(ind,tuple):
            nodeID = ind[0]
            forward = ind[1]
        else:
            nodeID = ind
            forward = True
            
        if nodeID>len(self.nodes):
            raise IndexError('The nodeID index {nodeID} is out of range')
            
        if not forward and self.isBack:
            try:
                return self.backLinks[nodeID]
            except KeyError:
                return None
        else:
            try:
                return self.forwardLinks[nodeID]
            except KeyError:
                return None